In [26]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- Input static variables ---
print("Enter the constant parameters:")

rho_O = float(input("Oil density (kg/m³) [800-1100]: "))
rho_W = float(input("Water density (kg/m³) [1000-1500]: "))
eff_pump = float(input("Injection pump efficiency (%) [75-85]: ")) / 100
eff_poly = float(input("Polymer efficiency (%) [0-100]: ")) / 100
eff_pui = float(input("Preparation unit efficiency (%) [75-90]: ")) / 100
d_shipping = float(input("Polymer transport distance (km) [1000-20000]: "))
H = float(input("Lift system height (m) [100-10000]: "))
ALS = float(input("ALS efficiency (%) [75-90]: ")) / 100
n_valves = int(input("Number of valves installed (1-15): "))
CO2_factor = float(input("CO2 emission factor (kg CO2 / kWh): "))
energy_cost = float(input("Energy cost (USD / kWh): "))

# Constants
X_trat = 5  # kW/m³
X_Oil_Chem = 45.63e6  # J/kg
X_man = 123.6e6  # J/kg
X_shipping = 188  # J/(kg·km)
g = 9.81  # m/s²
bbl_to_m3 = 0.158987
psi_to_Pa = 6894.76

# Load data
df = pd.read_excel("Wells_data.xlsx")

# Unit conversions
df["Qinj_m3"] = df["Qinj_B"] * bbl_to_m3
df["qoil_m3"] = df["qoil_B"] * bbl_to_m3
df["ΔP_Pa"] = df["WHP_psi"] * psi_to_Pa

# Initialize result columns
df["X_mix_J"] = 0.0
df["X_W_Prod_J"] = 0.0
df["X_Iny_J"] = 0.0
df["X_RFV_J"] = 0.0
df["X_ALS_J"] = 0.0
df["X_Oil_Total_J"] = 0.0
df["X_RF"] = 0.0

# Process each row
for i, row in df.iterrows():
    V_mix = row["Qinj_m3"]

    # X_mix
    X_mix = (V_mix * row["C"] * (X_man + X_shipping * d_shipping)) / eff_pui

    # X_W_Prod (Water treatment exergy)
    X_W_Prod = row["Qinj_m3"] * X_trat * 3600 * 1000  # convert kW·h to J

    # X_Iny (Injection pump exergy)
    X_Iny = row["Qinj_m3"] * row["ΔP_Pa"] / (eff_pump * eff_poly)

    # X_RFV (Valve exergy sum for all valves)
    X_RFV = 0
    for _ in range(n_valves):
        X_RFV += row["Qinj_m3"] * row["ΔP_Pa"] / eff_poly  # valve exergy per valve
    X_RFV /= eff_pump

    # X_ALS (Artificial lift system exergy)
    rho_mix = row["WOR"] * rho_W + (1 - row["WOR"]) * rho_O
    V_liq = row["qoil_m3"] * (1 + row["WOR"])
    X_ALS = V_liq * rho_mix * g * H / ALS

    # X_Oil_Total (Total chemical exergy of oil)
    m_oil = row["qoil_m3"] * rho_O
    X_Oil_Total = X_Oil_Chem * m_oil

    # Exergetic efficiency
    total_input_exergy = X_mix + X_W_Prod + X_Iny + X_RFV + X_ALS
    X_RF = (X_Oil_Total - total_input_exergy) / X_Oil_Total if X_Oil_Total != 0 else 0

    # Save results
    df.loc[i, "X_mix_J"] = X_mix
    df.loc[i, "X_W_Prod_J"] = X_W_Prod
    df.loc[i, "X_Iny_J"] = X_Iny
    df.loc[i, "X_RFV_J"] = X_RFV
    df.loc[i, "X_ALS_J"] = X_ALS
    df.loc[i, "X_Oil_Total_J"] = X_Oil_Total
    df.loc[i, "X_RF"] = X_RF

# Sum input exergies for emissions and cost
df["Input_Exergies_J"] = df["X_mix_J"] + df["X_W_Prod_J"] + df["X_Iny_J"] + df["X_RFV_J"] + df["X_ALS_J"]

# Convert input exergy to kWh (1 J = 2.77778e-7 kWh)
df["Input_Exergies_kWh"] = df["Input_Exergies_J"] * 2.77778e-7

# CO2 emissions in tons
df["CO2_Emissions_tons"] = df["Input_Exergies_kWh"] * CO2_factor / 1000

# Energy cost in thousand USD
df["Energy_Cost_kUSD"] = df["Input_Exergies_kWh"] * energy_cost / 1000

# Useful oil exergy fraction in GJ
df["Useful_Oil_Exergy_GJ"] = (df["X_Oil_Total_J"] * df["X_RF"]) / 1e9

# --- Plot 1: CO2 Emissions Over Time ---
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=df["Fecha_poly"],
    y=df["CO2_Emissions_tons"],
    mode="lines+markers",
    name="CO2 Emissions (tons)",
    line=dict(color="crimson"),
))
fig1.update_layout(
    title="CO₂ Emissions Over Time",
    xaxis_title="Date",
    yaxis_title="CO₂ Emissions (tons)",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    yaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    legend=dict(x=0.01, y=0.99, bgcolor="white"),
    width=720,
    height=480,
)
fig1.show()

# --- Plot 2: Energy Cost Over Time ---
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=df["Fecha_poly"],
    y=df["Energy_Cost_kUSD"],
    mode="lines+markers",
    name="Energy Cost (thousand USD)",
    line=dict(color="navy"),
))
fig2.update_layout(
    title="Energy Cost Over Time",
    xaxis_title="Date",
    yaxis_title="Energy Cost (thousand USD)",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    yaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    legend=dict(x=0.01, y=0.99, bgcolor="white"),
    width=720,
    height=480,
)
fig2.show()

# --- Plot 3: Exergetic Efficiency (X_RF) ---
fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=df["Fecha_poly"],
    y=df["X_RF"],
    mode="lines+markers",
    name="Exergetic Efficiency (X_RF)",
    line=dict(color="darkgreen"),
))
fig3.update_layout(
    title="Exergetic Efficiency (X_RF) Over Time",
    xaxis_title="Date",
    yaxis_title="X_RF",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    yaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    legend=dict(x=0.01, y=0.99, bgcolor="white"),
    width=720,
    height=480,
)
fig3.show()

# --- Plot 4: Useful Oil Exergy Over Time ---
fig4 = go.Figure()
fig4.add_trace(go.Scatter(
    x=df["Fecha_poly"],
    y=df["Useful_Oil_Exergy_GJ"],
    mode="lines+markers",
    name="Useful Oil Exergy (GJ)",
    line=dict(color="darkorange"),
))
fig4.update_layout(
    title="Useful Exergy of Produced Oil Over Time",
    xaxis_title="Date",
    yaxis_title="Useful Exergy (GJ)",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    yaxis=dict(showgrid=True, zeroline=True, zerolinecolor="lightgray"),
    legend=dict(x=0.01, y=0.99, bgcolor="white"),
    width=720,
    height=480,
)
fig4.show()






# Save results
df.to_excel("Exergy_CO2_EnergyCost_Results.xlsx", index=False)


Enter the constant parameters:
